In [4]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
from ipynb.fs.full import functions
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
ticker_excel_book = pd.read_excel('tickers.xlsx', sheet_name=0, index_col=0)
ticker_prices_excel_book = pd.read_excel('ticker_prices_transformed.xlsx', sheet_name=None, header=[0,1], index_col=0,)
market_prices_excel_book = pd.read_excel('market_prices_transformed.xlsx', sheet_name=None, index_col=0)

In [6]:
period_minus_10 = -10
period_minus_3 = -3
period_minus_1 = -1
period_zero = 0
period_plus_1 = 1
period_plus_20 = 20

u_after_event_gathered_list = []
u_event_gathered_list = []
leak_gathered_list = []
vto_gathered_list = []
mv_gathered_list = []
ret_gathered_list = []
vol_gathered_list = []

for case in market_prices_excel_book.keys ():

    ticker_df = ticker_excel_book.copy()
    ticker_prices_df = ticker_prices_excel_book[case].copy()['price']
    market_prices_df = market_prices_excel_book[case].copy()['price']
    ticker_volume_df = ticker_prices_excel_book[case].copy()['volume'].add(1)

    period_minus_10_iloc = market_prices_df.index.get_loc(period_minus_10)
    period_minus_3_iloc = market_prices_df.index.get_loc(period_minus_3)
    period_minus_1_iloc = market_prices_df.index.get_loc(period_minus_1)
    period_zero_iloc = market_prices_df.index.get_loc(period_zero)
    period_plus_1_iloc = market_prices_df.index.get_loc(period_plus_1)
    period_plus_20_iloc = market_prices_df.index.get_loc(period_plus_20)

    ticker_shares_series = ticker_df.copy()['shares'].sort_index()

    ticker_return_df = ticker_prices_df.copy().pct_change()
    market_return_df = market_prices_df.copy().pct_change()
    ticker_u_df = ticker_return_df.copy().sub(market_return_df.copy(), axis='index')
    volume_turnover_df = ticker_volume_df.copy().add(1).pct_change()

    u_after_event_df = ticker_u_df.copy().iloc[period_plus_1_iloc:period_plus_20_iloc+1]
    u_event_series = ticker_u_df.copy().iloc[period_zero_iloc]
    leak_series = ticker_u_df.iloc[period_minus_3_iloc:period_minus_1_iloc+1].sum()
    vto_df = volume_turnover_df.copy().iloc[period_plus_1_iloc:period_plus_20_iloc+1]
    mv_df = ticker_prices_df.copy().iloc[period_plus_1_iloc:period_plus_20_iloc+1]\
        .mul(ticker_shares_series.copy(), axis='columns')
    ret_series = ticker_return_df.loc[-10:-1].mean()
    vol_series = ticker_return_df.loc[-10:-1].var()

    u_after_event_gathered_list.append({
        'sheet_name': case,
        'data': u_after_event_df,
    })
    u_event_gathered_list.append({
        'sheet_name': case,
        'data': u_event_series,
    })
    leak_gathered_list.append({
        'sheet_name': case,
        'data': leak_series,
    })
    vto_gathered_list.append({
        'sheet_name': case,
        'data': vto_df,
    })
    mv_gathered_list.append({
        'sheet_name': case,
        'data': mv_df,
    })
    ret_gathered_list.append({
        'sheet_name': case,
        'data': ret_series,
    })
    vol_gathered_list.append({
        'sheet_name': case,
        'data': vol_series,
    })


In [7]:
with pd.ExcelWriter('gathered_u_after_event.xlsx') as excel_writer:
    for i in u_after_event_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('gathered_u_event.xlsx') as excel_writer:
    for i in u_event_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('gathered_leak.xlsx') as excel_writer:
    for i in leak_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('gathered_vto.xlsx') as excel_writer:
    for i in vto_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('gathered_mv.xlsx') as excel_writer:
    for i in mv_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('gathered_ret.xlsx') as excel_writer:
    for i in ret_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('gathered_vol.xlsx') as excel_writer:
    for i in vol_gathered_list:
        sheet_name = i['sheet_name']
        data = i['data']
        data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)
